# LLM-Chat test

In [1]:
api_key = ""  # 请填写您自己的APIKey

In [44]:
from zhipuai import ZhipuAI
client = ZhipuAI(api_key=api_key)  # 请填写您自己的APIKey
response = client.chat.completions.create(
    model="glm-4-flash",  # 请填写您要调用的模型名称
    messages=[
        {"role": "user", "content": "作为一名营销专家，请为我的产品创作一个吸引人的口号"},
        {"role": "assistant", "content": "当然，要创作一个吸引人的口号，请告诉我一些关于您产品的信息"},
        {"role": "user", "content": "智谱AI开放平台"},
        {"role": "assistant", "content": "点燃未来，智谱AI绘制无限，让创新触手可及！"},
        {"role": "user", "content": "创作一个更精准且吸引人的口号"}
    ],
)
print(response.choices[0].message)

CompletionMessage(content='智启未来，谱绘无限可能——智谱AI，让智慧触手可及！', role='assistant', tool_calls=None)


# Tools make

## web search tool

### web search api test

In [45]:
import requests
import uuid
import json

In [46]:
def run_v4_sync():
    msg = [
        {
            "role": "user",
            "content":"中国队奥运会拿了多少奖牌"
        }
    ]
    tool = "web-search-pro"
    url = "https://open.bigmodel.cn/api/paas/v4/tools"
    request_id = str(uuid.uuid4())
    data = {
        "request_id": request_id,
        "tool": tool,
        "stream": True,
        "messages": msg
    }

    resp = requests.post(
        url,
        json=data,
        headers={'Authorization': api_key},
        timeout=300
    )
    print(resp.content.decode())



if __name__ == '__main__':
    run_v4_sync()

data: {"id":"20250106112858d31019a583f44569","created":1736134138201,"model":"web-search-pro","choices":[{"index":0,"delta":{"role":"tool","tool_calls":[{"id":"20250106112858d31019a583f44569","type":"search_intent","search_intent":[{"index":0,"query":"中国队奥运会拿了多少奖牌","intent":"SEARCH_ALL","keywords":"中国队 奥运会 奖牌","category":""}]}]}}]}

data: {"id":"20250106112858d31019a583f44569","created":1736134138584,"model":"web-search-pro","choices":[{"index":0,"finish_reason":"stop","delta":{"role":"tool","tool_calls":[{"id":"20250106112858d31019a583f44569","type":"search_result","search_result":[{"index":0,"title":"人民体谈：2024“成绩单”耀眼，体育强国建设迈向新高度（发布时间：2024-12-31 15:09:17）","link":"https://sports.eastday.com/a/241231144318157706881.html","content":"从领跑奥运奖牌榜，到全民涌动健身潮；从经典优势项目稳固领先，到新兴小众运动多点绽放2024年，锚定建设体育强国目标，中国体育全方位展现出蓬勃力量，昂首阔步走在通向体育强国的康庄大道上。\n成绩单”耀眼，竞技体育迈向新高度。国家体育总局竞技体育司发布的《2024年我国运动员取得成绩报告》显示，今年我国运动员共在33个项目上获得194个世界冠军，其中，奥项世界冠军84个、非奥项世界冠军110个，数量创历年之最，同时创造19项世界纪录，一张张熠熠生辉的成绩单”，绘就2024体育领域精彩画卷。\n回眸2024年体坛，最

### web search api generate&parse

In [47]:
def web_search_api(query):
    msg = [
        {
            "role": "user",
            "content":query
        }
    ]
    tool = "web-search-pro"
    url = "https://open.bigmodel.cn/api/paas/v4/tools"
    request_id = str(uuid.uuid4())
    data = {
        "request_id": request_id,
        "tool": tool,
        "stream": False,
        "messages": msg
    }
    resp = requests.post(
        url,
        json=data,
        headers={'Authorization': api_key},
        timeout=300
    )
    json_string = resp.content.decode()
    json_data = json.loads(json_string)
    search_intent = json_data['choices'][0]['message']['tool_calls'][0]['search_intent']
    # print(search_intent)
    search_result = json_data['choices'][0]['message']['tool_calls'][1]['search_result']
    # for item in search_result:
    #     print(item)
    return search_result



### tools make

In [48]:
tools = [
    {
        "type": "function",
        "function": {
            "name": "web_search_api",
            "description": "根据原问题，从网上查询与原问题相关的信息",
            "parameters": {
                "type": "object",
                "properties": {
                    "query": {
                        "description": "原问题",
                        "type": "string"
                    },
                },
                "required": [ "query" ]
            },
        }
    }
]

### tools parse function

In [49]:
def parse_function_call(model_response,messages):
    # 处理函数调用结果，根据模型返回参数，调用对应的函数。
    # 调用函数返回结果后构造tool message，再次调用模型，将函数结果输入模型
    # 模型会将函数调用结果以自然语言格式返回给用户。
    if model_response.choices[0].message.tool_calls:
        tool_call = model_response.choices[0].message.tool_calls[0]
        args = tool_call.function.arguments
        function_result = {}
        if tool_call.function.name == "web_search_api":
            function_result = web_search_api(**json.loads(args))
        messages.append({
            "role": "tool",
            "content": f"{json.dumps(function_result)}",
            "tool_call_id":tool_call.id
        })
        response = client.chat.completions.create(
            model="glm-4-flash",  # 填写需要调用的模型名称
            messages=messages,
            tools=tools,
        )
        # print(response.choices[0].message)
        print(response.choices[0].message.content)
        messages.append(response.choices[0].message.model_dump())

## web get tool

### text split

In [26]:
import jieba
import jieba.analyse

def get_keywords_chinese(text):
    # 使用 jieba 提取关键词
    keywords = jieba.analyse.extract_tags(text, topK=5)
    return keywords

# 示例文本
text = "自然语言处理使计算机能够理解人类的语言。这个领域正在迅速发展。"
print(get_keywords_chinese(text))


['自然语言', '计算机', '语言', '理解', '人类']


# LLM Generate

In [50]:
query = '邯郸今天天气怎么样'

In [51]:
from zhipuai import ZhipuAI
client = ZhipuAI(api_key=api_key)  # 请填写您自己的APIKey
messages = []
messages.append({"role": "system", "content": "你是一个能进行web search的agent,不要假设或猜测传入函数的参数值。如果用户的描述不明确，请要求用户提供必要信息"})
messages.append({"role": "user", "content": query})

response = client.chat.completions.create(
    model="glm-4-flash",  # 请填写您要调用的模型名称
    messages=messages,
    tools=tools
)
print(response.choices[0].message)
messages.append(response.choices[0].message.model_dump())
# print(response.choices[0].message.model_dump())
parse_function_call(response,messages)

CompletionMessage(content=None, role='assistant', tool_calls=[CompletionMessageToolCall(id='call_-9095618577474462898', function=Function(arguments='{"query": "邯郸今天天气"}', name='web_search_api'), type='function', index=0)])
根据查询结果，邯郸今天天气为晴，气温5.2℃，东北风2级，湿度20%，空气质量为46。
